In [1]:
!hostname

HPP-LNR-047


# Imports/Setpu

In [ ]:
# import sys
# import os
# import shutil

# username = os.environ.get('USER')
# scratch_base = f"/scratch/midway3/{username}/fea_project"
# custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
# py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
# site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")

# # Clean up old installation
# if os.path.exists(custom_lib_path):
#     print(f"Removing old libraries...")
#     shutil.rmtree(custom_lib_path)

# install_cmd = (
#     f"pip install --target='{site_packages}' "
#     "--upgrade "
#     "--no-cache-dir "
#     "'numpy<2.0' "
#     "'pandas' "
#     "'bottleneck' "
#     "'numexpr' "
#     "'torch' "
#     "'torchvision' "
#     "'accelerate>=0.31.0' "
#     "'tensorflow' "
#     "'tf-keras' "
#     "'transformers' "
#     "'sentence-transformers' "
#     "'datasets' "
#     "'botocore' "
#     "'aiobotocore' "
#     "'s3fs' "
#     "'openpyxl' "
#     "'sentencepiece' "
#     "'plotly' "
#     "'scikit-learn' "
#     "'papermill' "
#     "'scrapbook' "
#     "'openai' "
#     "'pydantic' "
#     "'tqdm' "
# )

# print("Installing with NumPy <2.0...")
# exit_code = os.system(install_cmd)

# if exit_code == 0:
#     print("\nSUCCESS: Complete stack installed.")
# else:
#     print(f"\nERROR: Installation failed with code {exit_code}")

# print("\n##### RESTART KERNEL NOW #####")

In [1]:
import sys
import os

### Installs all the packages in the install library (my_custom_libs) ##

# --- 1. SETUP PATHS FIRST (you are going to need to change to whatever your path to the instal library is in) ---
username = os.environ.get('USER')
scratch_base = f"/scratch/midway3/{username}/fea_project"
custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
hf_cache_path = os.path.join(scratch_base, "hf_cache")

py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")
sys.path.insert(0, site_packages)

# FORCE custom path to the FRONT of the list
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

print(f"Python is looking here first: {sys.path[0]}")

# --- 2. ENV VARIABLES ---
os.environ['HF_HOME'] = hf_cache_path
os.environ['TRANSFORMERS_CACHE'] = hf_cache_path
os.environ['PYTHONUSERBASE'] = custom_lib_path

# --- 3. NOW IMPORT LIBRARIES ---
# Only import AFTER sys.path is updated
import datasets 
import sentence_transformers
from sentence_transformers import SentenceTransformer

# --- 4. VERIFY PATHS ---
print(f"Datasets is loaded from: {os.path.dirname(datasets.__file__)}")

if "software" in os.path.dirname(datasets.__file__):
    print("FAILURE: Still loading system datasets.")
else:
    print("SUCCESS: All libraries isolated in scratch.")

Python is looking here first: /scratch/midway3/None/fea_project\my_custom_libs\lib\python3.13\site-packages


KeyboardInterrupt: 

In [1]:
import sys
import os
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"Added current directory to sys.path: {current_dir}")
import papermill as pm
import scrapbook as sb
import pandas as pd
import os
from datetime import datetime
import importlib
import free_entailments_algorithm_utils as fea

Added current directory to sys.path: c:\Users\aesteva\Dropbox\Culture\3_data_processing\10_Argumentation\Entailment\CODE\free_entailment_algorithm\fea_project


# Global Variables:

In [2]:
#Change to False if running the real thing.

test = False

In [3]:
#Paths
importlib.reload(fea)
df_p = pd.read_excel(r"C:\Users\aesteva\Dropbox\Culture\3_data_processing\10_Argumentation\Entailment\DATA\round2_extra_speeches\Jan2026\final\ClauseLevel_df_p.xlsx")
df_sc = pd.read_excel(r"C:\Users\aesteva\Dropbox\Culture\3_data_processing\10_Argumentation\Entailment\DATA\round2_extra_speeches\Jan2026\final\ClauseLevel_df_sc.xlsx")

In [4]:
import gc

## All clauses. Of the form: argument_id	sentence_id	sentence

df_clause = pd.concat([df_p,df_sc]).drop_duplicates(subset='sentence_id')

# Generate initial unlabeled pairs (production mode only).
# We generate up to 50M pairs (BB + BS), excluding any already-labeled pairs.
# These are randomly sampled from the ~450M valid candidate space.
# seen_indices_dir persists consumed pair indices across rounds so
# subsequent rounds never re-sample pairs from earlier rounds.

if not test:
    df_pairs = fea.generate_valid_pairs(
        df_p,
        df_sc,
        'sentence_id',
        'sentence',
        max_pairs=5_000_000,
        exclude_labeled_csv="labeled_pairs/llm_labeled_pairs.csv",
        seen_indices_dir="seen_pair_indices",
    )
    print(f"Generated {len(df_pairs):,} unlabeled pairs (after excluding already-labeled)")

# Free source DataFrames — all data preserved in df_clause and df_pairs
del df_p, df_sc
gc.collect()


=== Generating premise-premise pairs ===
  Dataset size: 29635 (9564 books, 20071 speeches)
  Max possible valid pairs: 237,689,310
  Seen-index tracking: 0 BB + 0 BS already consumed (45,730,266 BB + 191,959,044 BS remaining)
  Sampling 3,835,253 pairs from 237,689,310 possible...
    BB: sampled 737,884 pairs (seen 0, avail 45,730,266)
    BS: sampled 3,097,369 pairs (seen 0, avail 191,959,044)
  Generated 3,835,253 unique pairs

=== Generating conclusion-conclusion pairs ===
  Dataset size: 9000 (2511 books, 6489 speeches)
  Max possible valid pairs: 19,445,184
  Seen-index tracking: 0 BB + 0 BS already consumed (3,151,305 BB + 16,293,879 BS remaining)
  Sampling 1,164,747 pairs from 19,445,184 possible...
    BB: sampled 188,760 pairs (seen 0, avail 3,151,305)
    BS: sampled 975,987 pairs (seen 0, avail 16,293,879)
  Generated 1,164,747 unique pairs

=== SUMMARY ===
Premise-premise pairs: 3,835,253
Conclusion-conclusion pairs: 1,164,747
Total valid pairs: 5,000,000
Columns: ['id1

9241

In [ ]:
# # Creates Embedding Cache (If not already created) from df_clause:
# importlib.reload(fea)

# import pickle

# cache_path = "embedding_cache_finetuned2.pkl"

# if os.path.exists(cache_path):
#     with open(cache_path, 'rb') as f:
#         embedding_cache_finetuned = pickle.load(f)
#     print(f"Loaded existing embedding cache: {len(embedding_cache_finetuned)} embeddings")
# else:
#     # Create cache from all unique clauses
#     embedding_cache_finetuned = fea.create_embedding_cache(
#     df_texts=df_clause,
#     id_col='sentence_id',
#     text_col='sentence',
#     model_name="C:\\Users\\aesteva\\Dropbox\\Culture\\3_data_processing\\10_Argumentation\\Entailment\\CODE\\free_entailment_algorithm\\fea_project\\fine_tuned_bi_model",
#     batch_size=128,
#     show_progress_bar=True
# )
#     # Save to disk for future runs
#     with open(cache_path, 'wb') as f:
#         pickle.dump(embedding_cache_finetuned, f)
#     print(f"Saved embedding cache to {cache_path}")


Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]


>>> Creating Embedding Cache
Model: C:\Users\aesteva\Dropbox\Culture\3_data_processing\10_Argumentation\Entailment\CODE\free_entailment_algorithm\fea_project\fine_tuned_bi_model
Total unique texts: 37474
Device: cpu


Batches:   0%|          | 0/293 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
import gc

input_file = "labeled_pairs/Results_DS_BtoS_iteration_0.csv"
df_input = pd.read_csv(input_file)

if test:
    importlib.reload(fea)
    df_input = pd.read_csv(input_file)
    
    input_file, remaining_llm_calls = fea.two_random_subsamples(
        df = df_input,
        frac1 = 0.8,
        frac2 = 0.2,
        random_state = 42
    )
else:
    remaining_llm_calls = None
    # Use Results_DS_BtoS_iteration_0 as labeled input,
    # and subtract already-labeled pairs from the 20M unlabeled pool
    df_entailed_pairs = df_input.rename(columns={'sentence_id_1': 'id1', 'sentence_id_2': 'id2'})
    unlabeled_pairs = fea.setminus(df_pairs, df_entailed_pairs, ["id1", "id2"])
    print(f"Unlabeled pairs after removing labeled: {len(unlabeled_pairs):,}")

    # Free the 75M-row df_pairs — unlabeled_pairs is the only copy we need.
    # This saves ~3 GB of RAM in the FEA_Loop kernel, which persists while
    # FEA_Pipeline and FreeEntailmentAlgorithm run in their own sub-processes.
    del df_pairs, df_entailed_pairs
    gc.collect()
    print("✓ Freed df_pairs (only unlabeled_pairs kept in memory)")

Set difference: 4,999,977 - 1,998 = 4,999,977 rows
Unlabeled pairs after removing labeled: 4,999,977
✓ Freed df_pairs (only unlabeled_pairs kept in memory)


In [6]:
import pickle
with open("embedding_cache_finetuned.pkl", 'rb') as f:
    embedding_cache_finetuned = pickle.load(f)
print(f"Loaded embedding cache: {len(embedding_cache_finetuned)} embeddings")

Loaded embedding cache: 38635 embeddings


In [7]:
num_iterations = 2          # hard cap (set high – budget is the real limiter)
iteration_stats = []

sent_frac = 0.5
budget = 5.00

# ---- Budget controls ----
# The loop stops automatically once cumulative LLM spend reaches budget_dollars.
# Set to None for unlimited (loop runs until num_iterations).
budget_dollars = 5.00       
cost_per_1k_pairs = 2.0       # ~$2 per 1,000 pairs (DeepSeek R1)

# DeepSeek API key for production LLM calls
api_key = os.environ.get("DEEPSEEK_API_KEY", "sk-")

# FEA Loop using Papermill

This notebook uses papermill to iteratively run the FEA_Pipeline notebook, feeding the output of each iteration as input to the next.

In [8]:
output_dir = "fea_iterations"
os.makedirs(output_dir, exist_ok=True)

start_iteration = 0

In [9]:
print(f"Starting FEA Loop with {num_iterations} iterations")
print("="*60)


Starting FEA Loop with 2 iterations


In [10]:
importlib.reload(fea)

iteration_stats = fea.run_fea_loop(
    test=test,
    input_file=input_file,
    df_clause=df_clause,
    embedding_cache=embedding_cache_finetuned,
    num_iterations=num_iterations,
    start_iteration=start_iteration,
    output_dir=output_dir,
    sent_frac=sent_frac,
    budget=budget,
    remaining_llm_calls=remaining_llm_calls,
    unlabeled_pairs=unlabeled_pairs if 'unlabeled_pairs' in dir() else None,
    deepseek_api_key=api_key,
    budget_dollars=budget_dollars,
    cost_per_1k_pairs=cost_per_1k_pairs,
)

VALIDATING INITIAL DATA STATE
Initial labeled pairs: 1998
Initial unlabeled pairs: 4999977
Total: 5001975
✓ Cleaned up old loop data directory
Starting with 1998 already labeled pairs
Unlabeled pool: 4999977
Total pairs: 5001975
Budget: $5.00  ($2.00 / 1k pairs)

ITERATION 0/2
Status: Labeled=1998, Unlabeled=4999977, Total=5001975
Cost so far: $0.00  ($5.00 remaining)
Executing FEA_Pipeline.ipynb with input: labeled_pairs/Results_DS_BtoS_iteration_0.csv


Executing:   0%|          | 0/26 [00:00<?, ?cell/s]


✓ Retrieved df_to_llm from scrapbook
  Shape: (1000, 7)
  Columns: ['sentence_id_2', 'sentence_id_1', 'sentence_text_2', 'argument_id_2', 'sentence_text_1', 'argument_id_1', 'score']
✓ Saved df_to_llm to fea_iterations\loop_data/df_to_llm_iter_0.csv

EXECUTING LLM EVALUATOR (forward direction)
Sending 1000 pairs to LLM...


Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

✓ Forward LLM evaluation complete
✓ LLM labeled pairs updated: 2997 total in labeled_pairs/llm_labeled_pairs.csv

ONE-WAY RESULTS PROCESSING
Total input pairs: 1000
Resolved rows: 1938 (YES=0, NO=1938)
  ↳ Inferred reverse NO (money saved): 969
Need reverse LLM call: 20

✓ LLM labeled pairs updated: 3966 total in labeled_pairs/llm_labeled_pairs.csv
✓ Recorded 969 inferred reverse-NO pairs in labeled_pairs/llm_labeled_pairs.csv

SENDING 20 REVERSE PAIRS TO LLM
Model: deepseek-reasoner
Input: labeled_pairs\reverse_iter_0_input.csv
Output: labeled_pairs\reverse_iter_0_output.csv
Loading data from labeled_pairs\reverse_iter_0_input.csv...
Loading data from ArgLevel_ClauseIds_df.xlsx...
Loaded 20 sentence pairs
Using model: deepseek-reasoner
Using prompt type: test_prompt_tot_json2
Running batch evaluation...


...

✓ Forward LLM evaluation complete
✓ LLM labeled pairs updated: 4985 total in labeled_pairs/llm_labeled_pairs.csv

ONE-WAY RESULTS PROCESSING
Total input pairs: 1000
Resolved rows: 1920 (YES=0, NO=1920)
  ↳ Inferred reverse NO (money saved): 960
Need reverse LLM call: 37

✓ LLM labeled pairs updated: 5945 total in labeled_pairs/llm_labeled_pairs.csv
✓ Recorded 960 inferred reverse-NO pairs in labeled_pairs/llm_labeled_pairs.csv

SENDING 37 REVERSE PAIRS TO LLM
Model: deepseek-reasoner
Input: labeled_pairs\reverse_iter_1_input.csv
Output: labeled_pairs\reverse_iter_1_output.csv
Loading data from labeled_pairs\reverse_iter_1_input.csv...
Loading data from ArgLevel_ClauseIds_df.xlsx...
Loaded 37 sentence pairs
Using model: deepseek-reasoner
Using prompt type: test_prompt_tot_json2
Running batch evaluation...


100%|██████████| 37/37 [00:00<00:00, 48.45it/s]


[DEBUG] content length: 1547, reasoning_content length: 7069
[DEBUG] content preview: {
  "sentence_id_1": "S0024115003p",
  "sentence_id_2": "B0536006p",
  "answers": "NO, YES, NO",
  "reasoning": "1. NO: Statement 1 is a broad principle about political integrity, while Statement 2 is a specific claim about resisting unlawful authority. There is no logical necessity that the former 
[DEBUG] reasoning preview: We are given two arguments and two statements derived from them. Statement 1: "Safeguarding the integrity of the political system is important" comes from Argument 1. Statement 2: "This principle holds true regardless of the level of authority, as even subordinate magistrates can be opposed if they 
Saved results to labeled_pairs\reverse_iter_1_output.csv
✓ Reverse LLM evaluation complete
✓ LLM labeled pairs updated: 5982 total in labeled_pairs/llm_labeled_pairs.csv

REVERSE RESULTS SUMMARY
Total reverse pairs resolved: 74
  Bidirectional YES: 24
  Bidirectional NO:  50


BIDIREC

Executing:   0%|          | 0/26 [00:00<?, ?cell/s]

KeyboardInterrupt: 

In [11]:
stats_df = fea.save_iteration_stats(iteration_stats, output_dir)


Iteration Statistics:
Empty DataFrame
Columns: []
Index: []


KeyError: 'pairs_selected'

In [ ]:
# Build final output: verdict=YES pairs only, AB/BA duplicates collapsed to a single AB row.
# The last iteration's output_file contains all accumulated labeled pairs.
_final_csv = iteration_stats[-1]['output_file']
_df_all = pd.read_csv(_final_csv)

# Standardize column names
_id1_col = 'sentence_id_1' if 'sentence_id_1' in _df_all.columns else 'id1'
_id2_col = 'sentence_id_2' if 'sentence_id_2' in _df_all.columns else 'id2'

# Keep only YES verdicts
_df_yes = _df_all[_df_all['verdict'] == 'YES'].copy()

# Normalize pairs: always store (min, max) so AB and BA map to the same row
_a = _df_yes[_id1_col].astype(str)
_b = _df_yes[_id2_col].astype(str)
_df_yes['id1'] = np.where(_a <= _b, _a, _b)
_df_yes['id2'] = np.where(_a <= _b, _b, _a)
_df_yes['Verdict'] = 'YES'

df_out = _df_yes[['id1', 'id2', 'Verdict']].drop_duplicates().reset_index(drop=True)

df_out.to_excel('df_out.xlsx', index=False)
print(f"df_out: {len(df_out)} unique entailed pairs (from {len(_df_yes)} YES rows, {len(_df_all)} total)")
df_out.head(10)

In [25]:
import numpy as np
df_out = pd.read_csv(r"labeled_pairs\\Results_DS_BtoS_iteration_2.csv")
df_out = df_out[df_out['verdict'] == 'YES'].copy()
_id1_col = 'sentence_id_1' if 'sentence_id_1' in df_out.columns else 'id1'
_id2_col = 'sentence_id_2' if 'sentence_id_2' in df_out.columns else 'id2'
_a = df_out[_id1_col].astype(str)
_b = df_out[_id2_col].astype(str)
df_out['id1'] = np.where(_a <= _b, _a, _b)
df_out['id2'] = np.where(_a <= _b, _b, _a)
df_out['verdict'] = 'YES'

df_out = df_out[['id1', 'id2', 'verdict']].drop_duplicates().reset_index(drop=True)



In [29]:
df_out.head()
df_out.to_excel('df_out.xlsx', index=False)
